In [1]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
data_orig = pd.read_excel('drive/My Drive/Colab Notebooks/WisconsinDataset/Forms_merged.xlsx', 'form1')

In [3]:
data_orig.tail()

,EID,FormID,Flagged,Attempt,tot_time,iraw.1,iraw.2,iraw.3,iraw.4,iraw.5,iraw.6,iraw.7,iraw.8,iraw.9,iraw.10,iraw.11,iraw.12,iraw.13,iraw.14,iraw.15,iraw.16,iraw.17,iraw.18,iraw.19,iraw.20,iraw.21,iraw.22,iraw.23,iraw.24,iraw.25,iraw.26,iraw.27,iraw.28,iraw.29,iraw.30,iraw.31,iraw.32,iraw.33,iraw.34,iraw.35,...,idur.214,idur.215,idur.216,idur.217,idur.218,idur.219,idur.220,idur.221,idur.222,idur.223,idur.224,idur.225,idur.226,idur.227,idur.228,idur.229,idur.230,idur.231,idur.232,idur.233,idur.234,idur.235,idur.236,idur.237,idur.238,idur.239,idur.240,idur.241,idur.242,idur.243,idur.244,idur.245,idur.246,idur.247,idur.248,idur.249,idur.250,idur.251,idur.252,idur.253
3275,e201640,form2,0,1,10392,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,0,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,220.0,54.0,36.0,63.0,25.0,45.0,27.0,68.0,80.0,11.0,58.0,206.0,51.0,91.0,62.0,48.0,17.0,9.0,32.0,17.0,48.0,122.0,58.0,79.0,66.0,52.0,99.0,51.0,39.0,14.0,57.0,27.0,134.0,51.0,17.0,23.0,50.0,41.0,54.0,83.0
3276,e201641,form2,0,1,11202,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,63.0,58.0,82.0,49.0,17.0,42.0,35.0,35.0,46.0,52.0,129.0,181.0,83.0,125.0,44.0,33.0,51.0,32.0,55.0,61.0,36.0,22.0,68.0,115.0,50.0,29.0,56.0,46.0,44.0,37.0,65.0,30.0,92.0,36.0,24.0,23.0,25.0,42.0,62.0,58.0
3277,e201642,form2,0,1,13219,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,0,...,75.0,109.0,45.0,63.0,20.0,43.0,101.0,33.0,73.0,25.0,88.0,59.0,58.0,71.0,69.0,143.0,45.0,62.0,60.0,35.0,77.0,46.0,57.0,105.0,82.0,65.0,105.0,89.0,22.0,36.0,106.0,34.0,197.0,109.0,24.0,38.0,62.0,55.0,25.0,48.0
3278,e201643,form2,0,1,10205,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,75.0,24.0,32.0,60.0,14.0,53.0,23.0,33.0,31.0,22.0,99.0,149.0,39.0,53.0,23.0,56.0,82.0,29.0,37.0,53.0,58.0,21.0,48.0,78.0,49.0,51.0,42.0,127.0,48.0,57.0,65.0,33.0,75.0,65.0,29.0,26.0,115.0,88.0,49.0,58.0
3279,e201644,form2,0,1,6825,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,51.0,35.0,35.0,27.0,24.0,31.0,17.0,35.0,72.0,23.0,36.0,57.0,46.0,23.0,20.0,58.0,18.0,34.0,21.0,24.0,45.0,20.0,52.0,71.0,49.0,64.0,45.0,66.0,26.0,45.0,59.0,22.0,49.0,42.0,27.0,25.0,25.0,38.0,22.0,39.0


In [4]:
data_final = data_orig
data_final

,EID,FormID,Flagged,Attempt,tot_time,iraw.1,iraw.2,iraw.3,iraw.4,iraw.5,iraw.6,iraw.7,iraw.8,iraw.9,iraw.10,iraw.11,iraw.12,iraw.13,iraw.14,iraw.15,iraw.16,iraw.17,iraw.18,iraw.19,iraw.20,iraw.21,iraw.22,iraw.23,iraw.24,iraw.25,iraw.26,iraw.27,iraw.28,iraw.29,iraw.30,iraw.31,iraw.32,iraw.33,iraw.34,iraw.35,...,idur.214,idur.215,idur.216,idur.217,idur.218,idur.219,idur.220,idur.221,idur.222,idur.223,idur.224,idur.225,idur.226,idur.227,idur.228,idur.229,idur.230,idur.231,idur.232,idur.233,idur.234,idur.235,idur.236,idur.237,idur.238,idur.239,idur.240,idur.241,idur.242,idur.243,idur.244,idur.245,idur.246,idur.247,idur.248,idur.249,idur.250,idur.251,idur.252,idur.253
0,e100001,form1,0,5,10133,1.0,1.0,1.0,0,1.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0.0,0,1,1,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0,1.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e100002,form1,1,1,12409,1.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0.0,0,1,1,1.0,0.0,0.0,0.0,0.0,1,0.0,0,0,0.0,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e100003,form1,1,1,12457,0.0,0.0,0.0,0,1.0,0.0,1.0,0.0,0.0,0,0.0,1.0,1.0,1.0,0.0,1.0,1,1,0.0,0,0,1,0.0,0.0,1.0,0.0,0.0,0,1.0,0,0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e100004,form1,1,3,11389,1.0,1.0,0.0,0,1.0,0.0,0.0,0.0,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,1.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e100005,form1,1,5,13669,1.0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,1,1,1.0,1.0,1.0,0.0,0.0,1,0.0,0,1,1.0,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,e201640,form2,0,1,10392,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,0,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,220.0,54.0,36.0,63.0,25.0,45.0,27.0,68.0,80.0,11.0,58.0,206.0,51.0,91.0,62.0,48.0,17.0,9.0,32.0,17.0,48.0,122.0,58.0,79.0,66.0,52.0,99.0,51.0,39.0,14.0,57.0,27.0,134.0,51.0,17.0,23.0,50.0,41.0,54.0,83.0
3276,e201641,form2,0,1,11202,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,63.0,58.0,82.0,49.0,17.0,42.0,35.0,35.0,46.0,52.0,129.0,181.0,83.0,125.0,44.0,33.0,51.0,32.0,55.0,61.0,36.0,22.0,68.0,115.0,50.0,29.0,56.0,46.0,44.0,37.0,65.0,30.0,92.0,36.0,24.0,23.0,25.0,42.0,62.0,58.0
3277,e201642,form2,0,1,13219,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,0,...,75.0,109.0,45.0,63.0,20.0,43.0,101.0,33.0,73.0,25.0,88.0,59.0,58.0,71.0,69.0,143.0,45.0,62.0,60.0,35.0,77.0,46.0,57.0,105.0,82.0,65.0,105.0,89.0,22.0,36.0,106.0,34.0,197.0,109.0,24.0,38.0,62.0,55.0,25.0,48.0
3278,e201643,form2,0,1,10205,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,75.0,24.0,32.0,60.0,14.0,53.0,23.0,33.0,31.0,22.0,99.0,149.0,39.0,53.0,23.0,56.0,82.0,29.0,37.0,53.0,58.0,21.0,48.0,78.0,49.0,51.0,42.0,127.0,48.0,57.0,65.0,33.0,75.0,65.0,29.0,26.0,115.0,88.0,49.0,58.0


In [5]:
print(data_final.isnull().sum())
print(data_final.shape)

EID            0
FormID         0
Flagged        0
Attempt        0
tot_time       0
            ... 
idur.249    1636
idur.250    1636
idur.251    1636
idur.252    1636
idur.253    1636
Length: 511, dtype: int64
(3280, 511)


In [6]:
##### time #####
data_final['mean_idur'] = data_final.loc[:, 'idur.1':'idur.170'].mean(axis=1)
data_final['median_idur'] = data_final.loc[:, 'idur.1':'idur.170'].median(axis=1)
data_final['max_idur'] = data_final.loc[:, 'idur.1':'idur.170'].max(axis=1)
data_final['min_idur'] = data_final.loc[:, 'idur.1':'idur.170'].min(axis=1)

#### correct answers####
data_final['iraw_score'] = data_final.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
data_final['iraw_score_pct'] = data_final['iraw_score']/170
data_final

,EID,FormID,Flagged,Attempt,tot_time,iraw.1,iraw.2,iraw.3,iraw.4,iraw.5,iraw.6,iraw.7,iraw.8,iraw.9,iraw.10,iraw.11,iraw.12,iraw.13,iraw.14,iraw.15,iraw.16,iraw.17,iraw.18,iraw.19,iraw.20,iraw.21,iraw.22,iraw.23,iraw.24,iraw.25,iraw.26,iraw.27,iraw.28,iraw.29,iraw.30,iraw.31,iraw.32,iraw.33,iraw.34,iraw.35,...,idur.220,idur.221,idur.222,idur.223,idur.224,idur.225,idur.226,idur.227,idur.228,idur.229,idur.230,idur.231,idur.232,idur.233,idur.234,idur.235,idur.236,idur.237,idur.238,idur.239,idur.240,idur.241,idur.242,idur.243,idur.244,idur.245,idur.246,idur.247,idur.248,idur.249,idur.250,idur.251,idur.252,idur.253,mean_idur,median_idur,max_idur,min_idur,iraw_score,iraw_score_pct
0,e100001,form1,0,5,10133,1.0,1.0,1.0,0,1.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0.0,0,1,1,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0,1.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.323529,50.0,143.0,14.0,54.0,0.317647
1,e100002,form1,1,1,12409,1.0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0.0,0,1,1,1.0,0.0,0.0,0.0,0.0,1,0.0,0,0,0.0,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.158824,63.0,202.0,11.0,55.0,0.323529
2,e100003,form1,1,1,12457,0.0,0.0,0.0,0,1.0,0.0,1.0,0.0,0.0,0,0.0,1.0,1.0,1.0,0.0,1.0,1,1,0.0,0,0,1,0.0,0.0,1.0,0.0,0.0,0,1.0,0,0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.547059,65.5,179.0,16.0,61.0,0.358824
3,e100004,form1,1,3,11389,1.0,1.0,0.0,0,1.0,0.0,0.0,0.0,1.0,0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0.0,1.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.882353,62.0,154.0,12.0,61.0,0.358824
4,e100005,form1,1,5,13669,1.0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0,1,1,1.0,1.0,1.0,0.0,0.0,1,0.0,0,1,1.0,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.547059,69.0,189.0,0.0,62.0,0.364706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,e201640,form2,0,1,10392,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,0,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,27.0,68.0,80.0,11.0,58.0,206.0,51.0,91.0,62.0,48.0,17.0,9.0,32.0,17.0,48.0,122.0,58.0,79.0,66.0,52.0,99.0,51.0,39.0,14.0,57.0,27.0,134.0,51.0,17.0,23.0,50.0,41.0,54.0,83.0,53.597701,49.0,201.0,8.0,73.0,0.429412
3276,e201641,form2,0,1,11202,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,35.0,35.0,46.0,52.0,129.0,181.0,83.0,125.0,44.0,33.0,51.0,32.0,55.0,61.0,36.0,22.0,68.0,115.0,50.0,29.0,56.0,46.0,44.0,37.0,65.0,30.0,92.0,36.0,24.0,23.0,25.0,42.0,62.0,58.0,64.482759,52.0,189.0,13.0,73.0,0.429412
3277,e201642,form2,0,1,13219,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,0,...,101.0,33.0,73.0,25.0,88.0,59.0,58.0,71.0,69.0,143.0,45.0,62.0,60.0,35.0,77.0,46.0,57.0,105.0,82.0,65.0,105.0,89.0,22.0,36.0,106.0,34.0,197.0,109.0,24.0,38.0,62.0,55.0,25.0,48.0,71.413793,56.0,297.0,15.0,74.0,0.435294
3278,e201643,form2,0,1,10205,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,1,...,23.0,33.0,31.0,22.0,99.0,149.0,39.0,53.0,23.0,56.0,82.0,29.0,37.0,53.0,58.0,21.0,48.0,78.0,49.0,51.0,42.0,127.0,48.0,57.0,6

In [7]:
data_final.fillna(data_final.mean(), inplace=True)

In [8]:
######### iraw_idur_OEF (both+OEF) #############
col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct']

for x in range(1,171):
  col_features.append('idur.'+str(x))

for i in range(1,171):
  col_features.append('iraw.'+str(i))

X = data_final.loc[:, col_features]
Y = data_final.Flagged    

**End of Dataset Features selection!**

In [9]:
from collections import Counter
from matplotlib import pyplot
from numpy import where
counter = Counter(Y)
print(counter)

Counter({0: 3186, 1: 94})


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

In [11]:
import numpy as np

from sklearn.metrics import confusion_matrix

In [12]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


In [13]:

from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
from sklearn.metrics import auc

from numpy import sqrt
from numpy import argmax

In [14]:
###### SMOTE and RandomUnderSampler (for balancing data) ############

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [15]:
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [16]:
import xlwt 
from xlwt import Workbook 
  
wb = Workbook() 

In [17]:
def model_fit_proba(data_x, data_y, data_testx, kmodel):
    # Fit, Predict
    kmodel.fit(data_x, data_y)
    proba = kmodel.predict_proba(data_testx)
    proba = proba[:, 1]

    return proba

In [18]:

k_fold = model_selection.KFold(n_splits = 4)

full_trainset_x = np.array([])
full_trainset_y = np.array([])
full_testset_x = np.array([])
full_testset_y = np.array([])

full_testset_pred1 = np.array([])
full_testset_pred2 = np.array([])
full_testset_pred3 = np.array([])
full_testset_pred4 = np.array([])
full_testset_pred5 = np.array([])
full_testset_pred6 = np.array([])
full_testset_pred7 = np.array([])
full_testset_pred8 = np.array([])
full_testset_pred9 = np.array([])

full_testset_probs1 = np.array([])
full_testset_probs2 = np.array([])
full_testset_probs3 = np.array([])
full_testset_probs4 = np.array([])
full_testset_probs5 = np.array([])
full_testset_probs6 = np.array([])
full_testset_probs7 = np.array([])
full_testset_probs8 = np.array([])
full_testset_probs9 = np.array([])

nosmote_probs1 = np.array([])
nosmote_probs2 = np.array([])
nosmote_probs3 = np.array([])
nosmote_probs4 = np.array([])
nosmote_probs5 = np.array([])
nosmote_probs6 = np.array([])
nosmote_probs7 = np.array([])
nosmote_probs8 = np.array([])
nosmote_probs9 = np.array([])

nosmote_pred1 = np.array([])
nosmote_pred2 = np.array([])
nosmote_pred3 = np.array([])
nosmote_pred4 = np.array([])
nosmote_pred5 = np.array([])
nosmote_pred6 = np.array([])
nosmote_pred7 = np.array([])
nosmote_pred8 = np.array([])
nosmote_pred9 = np.array([])

precision1_1 = np.array([])
recall1_1 = np.array([])
f1_score1_1 = np.array([])
kappa1_1 = np.array([])
roc1_1 = np.array([])
fpr1_1 = np.array([])

precision1_2 = np.array([])
recall1_2 = np.array([])
f1_score1_2 = np.array([])
kappa1_2 = np.array([])
roc1_2 = np.array([])
fpr1_2 = np.array([])

precision1_3 = np.array([])
recall1_3 = np.array([])
f1_score1_3 = np.array([])
kappa1_3 = np.array([])
roc1_3 = np.array([])
fpr1_3 = np.array([])

precision1_4 = np.array([])
recall1_4 = np.array([])
f1_score1_4 = np.array([])
kappa1_4 = np.array([])
roc1_4 = np.array([])
fpr1_4 = np.array([])


precision2_1 = np.array([])
recall2_1 = np.array([])
f1_score2_1 = np.array([])
kappa2_1 = np.array([])
roc2_1 = np.array([])
fpr2_1 = np.array([])

precision2_2 = np.array([])
recall2_2 = np.array([])
f1_score2_2 = np.array([])
kappa2_2 = np.array([])
roc2_2 = np.array([])
fpr2_2 = np.array([])

precision2_3 = np.array([])
recall2_3 = np.array([])
f1_score2_3 = np.array([])
kappa2_3 = np.array([])
roc2_3 = np.array([])
fpr2_3 = np.array([])

precision2_4 = np.array([])
recall2_4 = np.array([])
f1_score2_4 = np.array([])
kappa2_4 = np.array([])
roc2_4 = np.array([])
fpr2_4 = np.array([])


precision3_1 = np.array([])
recall3_1 = np.array([])
f1_score3_1 = np.array([])
kappa3_1 = np.array([])
roc3_1 = np.array([])
fpr3_1 = np.array([])

precision3_2 = np.array([])
recall3_2 = np.array([])
f1_score3_2 = np.array([])
kappa3_2 = np.array([])
roc3_2 = np.array([])
fpr3_2 = np.array([])

precision3_3 = np.array([])
recall3_3 = np.array([])
f1_score3_3 = np.array([])
kappa3_3 = np.array([])
roc3_3 = np.array([])
fpr3_3 = np.array([])

precision3_4 = np.array([])
recall3_4 = np.array([])
f1_score3_4 = np.array([])
kappa3_4 = np.array([])
roc3_4 = np.array([])
fpr3_4 = np.array([])


precision4_1 = np.array([])
recall4_1 = np.array([])
f1_score4_1 = np.array([])
kappa4_1 = np.array([])
roc4_1 = np.array([])
fpr4_1 = np.array([])

precision4_2 = np.array([])
recall4_2 = np.array([])
f1_score4_2 = np.array([])
kappa4_2 = np.array([])
roc4_2 = np.array([])
fpr4_2 = np.array([])

precision4_3 = np.array([])
recall4_3 = np.array([])
f1_score4_3 = np.array([])
kappa4_3 = np.array([])
roc4_3 = np.array([])
fpr4_3 = np.array([])

precision4_4 = np.array([])
recall4_4 = np.array([])
f1_score4_4 = np.array([])
kappa4_4 = np.array([])
roc4_4 = np.array([])
fpr4_4 = np.array([])


precision5_1 = np.array([])
recall5_1 = np.array([])
f1_score5_1 = np.array([])
kappa5_1 = np.array([])
roc5_1 = np.array([])
fpr5_1 = np.array([])

precision5_2 = np.array([])
recall5_2 = np.array([])
f1_score5_2 = np.array([])
kappa5_2 = np.array([])
roc5_2 = np.array([])
fpr5_2 = np.array([])

precision5_3 = np.array([])
recall5_3 = np.array([])
f1_score5_3 = np.array([])
kappa5_3 = np.array([])
roc5_3 = np.array([])
fpr5_3 = np.array([])

precision5_4 = np.array([])
recall5_4 = np.array([])
f1_score5_4 = np.array([])
kappa5_4 = np.array([])
roc5_4 = np.array([])
fpr5_4 = np.array([])


precision6_1 = np.array([])
recall6_1 = np.array([])
f1_score6_1 = np.array([])
kappa6_1 = np.array([])
roc6_1 = np.array([])
fpr6_1 = np.array([])

precision6_2 = np.array([])
recall6_2 = np.array([])
f1_score6_2 = np.array([])
kappa6_2 = np.array([])
roc6_2 = np.array([])
fpr6_2 = np.array([])

precision6_3 = np.array([])
recall6_3 = np.array([])
f1_score6_3 = np.array([])
kappa6_3 = np.array([])
roc6_3 = np.array([])
fpr6_3 = np.array([])

precision6_4 = np.array([])
recall6_4 = np.array([])
f1_score6_4 = np.array([])
kappa6_4 = np.array([])
roc6_4 = np.array([])
fpr6_4 = np.array([])


precision7_1 = np.array([])
recall7_1 = np.array([])
f1_score7_1 = np.array([])
kappa7_1 = np.array([])
roc7_1 = np.array([])
fpr7_1 = np.array([])

precision7_2 = np.array([])
recall7_2 = np.array([])
f1_score7_2 = np.array([])
kappa7_2 = np.array([])
roc7_2 = np.array([])
fpr7_2 = np.array([])

precision7_3 = np.array([])
recall7_3 = np.array([])
f1_score7_3 = np.array([])
kappa7_3 = np.array([])
roc7_3 = np.array([])
fpr7_3 = np.array([])

precision7_4 = np.array([])
recall7_4 = np.array([])
f1_score7_4 = np.array([])
kappa7_4 = np.array([])
roc7_4 = np.array([])
fpr7_4 = np.array([])


precision8_1 = np.array([])
recall8_1 = np.array([])
f1_score8_1 = np.array([])
kappa8_1 = np.array([])
roc8_1 = np.array([])
fpr8_1 = np.array([])

precision8_2 = np.array([])
recall8_2 = np.array([])
f1_score8_2 = np.array([])
kappa8_2 = np.array([])
roc8_2 = np.array([])
fpr8_2 = np.array([])

precision8_3 = np.array([])
recall8_3 = np.array([])
f1_score8_3 = np.array([])
kappa8_3 = np.array([])
roc8_3 = np.array([])
fpr8_3 = np.array([])

precision8_4 = np.array([])
recall8_4 = np.array([])
f1_score8_4 = np.array([])
kappa8_4 = np.array([])
roc8_4 = np.array([])
fpr8_4 = np.array([])


i = 0
for train_ix, test_ix in k_fold.split(X_scaled):
    i = i + 1
    sheet = wb.add_sheet('fold'+str(i), cell_overwrite_ok=True) 

    sheet.write(0, 0, 'Model')
    sheet.write(0, 1, 'dataset')
    sheet.write(0, 2, 'threshold')
    sheet.write(0, 3, 'Precision')
    sheet.write(0, 4, 'Recall')
    sheet.write(0, 5, 'F1 Score')
    sheet.write(0, 6, 'weighted Kappa')
    sheet.write(0, 7, 'ROC AUC')
    sheet.write(0, 8, 'FPR')
    sheet.write(0, 9, 'tp')
    sheet.write(0, 10, 'fn')
    sheet.write(0, 11, 'fp')
    sheet.write(0, 12, 'tn')
    sheet.write(0, 13, 'total samples')
    sheet.write(0, 14, 'rate of predicted cheaters')

    # kfold
    train_x, train_y, test_x, test_y = X_scaled[train_ix], Y[train_ix], X_scaled[test_ix], Y[test_ix]

    # full_trainset_x = np.append(full_trainset_y, train_x)
    # full_trainset_y = np.append(full_trainset_y, train_y)
    # full_testset_x = np.append(full_testset_x, test_x)
    # full_testset_y = np.append(full_testset_y, test_y)

    ###### SMOTE and RandomUnderSampler (for balancing data) ############

    # both 33%
    # over = SMOTE(sampling_strategy=0.4)
    # under = RandomUnderSampler(sampling_strategy=0.5)
    # steps = [('o', over), ('u', under)]

    # both 50%
    # over = SMOTE(sampling_strategy=0.4)
    # under = RandomUnderSampler(sampling_strategy=1)
    # steps = [('o', over), ('u', under)]

    # only smote 50%
    # over = SMOTE(sampling_strategy=1)
    # steps = [('o', over)]

    # # only smote 33%
    # over = SMOTE(sampling_strategy=0.5)
    # steps = [('o', over)]

    # # only undersampling 50%
    # under = RandomUnderSampler(sampling_strategy=1)
    # steps = [('u', under)]

    # # only undersampling 33%
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('u', under)]

    pipeline = Pipeline(steps=steps)

    x_smote, y_smote = pipeline.fit_resample(train_x, train_y)

 ###### eight models #################
    ###
    model1 = DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best")
    # Fit, Predict
    model1.fit(x_smote, y_smote)
    probs1 = model1.predict_proba(test_x)
    probs1 = probs1[:, 1]

    pred1 = (probs1 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs1)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(1, 0, "Decision Tree")
    sheet.write(1, 1, "iraw_idur_OEF")
    sheet.write(1, 2, "0.7")
    sheet.write(1, 3, precision_score(test_y, pred1))
    sheet.write(1, 4, recall_score(test_y, pred1))
    sheet.write(1, 5, f1_score(test_y, pred1))
    sheet.write(1, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    sheet.write(1, 7, auc)
    sheet.write(1, 8, fpr)

    sheet.write(1, 9, str(tp))
    sheet.write(1, 10, str(fn))
    sheet.write(1, 11, str(fp))
    sheet.write(1, 12, str(tn))
    sheet.write(1, 13, str(tot_n))
    sheet.write(1, 14, str(fptp_rate))

    precision1_1 = np.append(precision1_1, precision_score(test_y, pred1))
    recall1_1 = np.append(recall1_1, recall_score(test_y, pred1))
    f1_score1_1 = np.append(f1_score1_1, f1_score(test_y, pred1))
    kappa1_1 = np.append(kappa1_1, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    roc1_1 = np.append(roc1_1, auc)
    fpr1_1 = np.append(fpr1_1, fpr)


    ###########
    pred1 = (probs1 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs1)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(2, 0, "Decision Tree")
    sheet.write(2, 1, "iraw_idur_OEF")
    sheet.write(2, 2, "0.5")
    sheet.write(2, 3, precision_score(test_y, pred1))
    sheet.write(2, 4, recall_score(test_y, pred1))
    sheet.write(2, 5, f1_score(test_y, pred1))
    sheet.write(2, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    sheet.write(2, 7, auc)
    sheet.write(2, 8, fpr)

    sheet.write(2, 9, str(tp))
    sheet.write(2, 10, str(fn))
    sheet.write(2, 11, str(fp))
    sheet.write(2, 12, str(tn))
    sheet.write(2, 13, str(tot_n))
    sheet.write(2, 14, str(fptp_rate))

    precision1_2 = np.append(precision1_2, precision_score(test_y, pred1))
    recall1_2 = np.append(recall1_2, recall_score(test_y, pred1))
    f1_score1_2 = np.append(f1_score1_2, f1_score(test_y, pred1))
    kappa1_2 = np.append(kappa1_2, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    roc1_2 = np.append(roc1_2, auc)
    fpr1_2 = np.append(fpr1_2, fpr)

    ###########
    pred1 = (probs1 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs1)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(3, 0, "Decision Tree")
    sheet.write(3, 1, "iraw_idur_OEF")
    sheet.write(3, 2, "0.97")
    sheet.write(3, 3, precision_score(test_y, pred1))
    sheet.write(3, 4, recall_score(test_y, pred1))
    sheet.write(3, 5, f1_score(test_y, pred1))
    sheet.write(3, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    sheet.write(3, 7, auc)
    sheet.write(3, 8, fpr)

    sheet.write(3, 9, str(tp))
    sheet.write(3, 10, str(fn))
    sheet.write(3, 11, str(fp))
    sheet.write(3, 12, str(tn))
    sheet.write(3, 13, str(tot_n))
    sheet.write(3, 14, str(fptp_rate))

    precision1_3 = np.append(precision1_3, precision_score(test_y, pred1))
    recall1_3 = np.append(recall1_3, recall_score(test_y, pred1))
    f1_score1_3 = np.append(f1_score1_3, f1_score(test_y, pred1))
    kappa1_3 = np.append(kappa1_3, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    roc1_3 = np.append(roc1_3, auc)
    fpr1_3 = np.append(fpr1_3, fpr)

    ###########
    pred1 = (probs1 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred1).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs1)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(4, 0, "Decision Tree")
    sheet.write(4, 1, "iraw_idur_OEF")
    sheet.write(4, 2, "0.03")
    sheet.write(4, 3, precision_score(test_y, pred1))
    sheet.write(4, 4, recall_score(test_y, pred1))
    sheet.write(4, 5, f1_score(test_y, pred1))
    sheet.write(4, 6, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    sheet.write(4, 7, auc)
    sheet.write(4, 8, fpr)

    sheet.write(4, 9, str(tp))
    sheet.write(4, 10, str(fn))
    sheet.write(4, 11, str(fp))
    sheet.write(4, 12, str(tn))
    sheet.write(4, 13, str(tot_n))
    sheet.write(4, 14, str(fptp_rate))

    precision1_4 = np.append(precision1_4, precision_score(test_y, pred1))
    recall1_4 = np.append(recall1_4, recall_score(test_y, pred1))
    f1_score1_4 = np.append(f1_score1_4, f1_score(test_y, pred1))
    kappa1_4 = np.append(kappa1_4, cohen_kappa_score(test_y, pred1, weights = 'quadratic'))
    roc1_4 = np.append(roc1_4, auc)
    fpr1_4 = np.append(fpr1_4, fpr)


############################    ###
    model2 = SVC(kernel='linear', probability=True)
    # Fit, Predict
    model2.fit(x_smote, y_smote)
    probs2 = model2.predict_proba(test_x)
    probs2 = probs2[:, 1]

    pred2 = (probs2 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs2)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(5, 0, "Support Vector Machine")
    sheet.write(5, 1, "iraw_idur_OEF")
    sheet.write(5, 2, "0.7")
    sheet.write(5, 3, precision_score(test_y, pred2))
    sheet.write(5, 4, recall_score(test_y, pred2))
    sheet.write(5, 5, f1_score(test_y, pred2))
    sheet.write(5, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    sheet.write(5, 7, auc)
    sheet.write(5, 8, fpr)
    sheet.write(5, 9, str(tp))
    sheet.write(5, 10, str(fn))
    sheet.write(5, 11, str(fp))
    sheet.write(5, 12, str(tn))
    sheet.write(5, 13, str(tot_n))
    sheet.write(5, 14, str(fptp_rate))

    precision2_1 = np.append(precision2_1, precision_score(test_y, pred2))
    recall2_1 = np.append(recall2_1, recall_score(test_y, pred2))
    f1_score2_1 = np.append(f1_score2_1, f1_score(test_y, pred2))
    kappa2_1 = np.append(kappa2_1, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    roc2_1 = np.append(roc2_1, auc)
    fpr2_1 = np.append(fpr2_1, fpr)


    ############
    pred2 = (probs2 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs2)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(6, 0, "Support Vector Machine")
    sheet.write(6, 1, "iraw_idur_OEF")
    sheet.write(6, 2, "0.5")
    sheet.write(6, 3, precision_score(test_y, pred2))
    sheet.write(6, 4, recall_score(test_y, pred2))
    sheet.write(6, 5, f1_score(test_y, pred2))
    sheet.write(6, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    sheet.write(6, 7, auc)
    sheet.write(6, 8, fpr)
    sheet.write(6, 9, str(tp))
    sheet.write(6, 10, str(fn))
    sheet.write(6, 11, str(fp))
    sheet.write(6, 12, str(tn))
    sheet.write(6, 13, str(tot_n))
    sheet.write(6, 14, str(fptp_rate))

    precision2_2 = np.append(precision2_2, precision_score(test_y, pred2))
    recall2_2 = np.append(recall2_2, recall_score(test_y, pred2))
    f1_score2_2 = np.append(f1_score2_2, f1_score(test_y, pred2))
    kappa2_2 = np.append(kappa2_2, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    roc2_2 = np.append(roc2_2, auc)
    fpr2_2 = np.append(fpr2_2, fpr)

   ############
    pred2 = (probs2 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs2)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(7, 0, "Support Vector Machine")
    sheet.write(7, 1, "iraw_idur_OEF")
    sheet.write(7, 2, "0.97")
    sheet.write(7, 3, precision_score(test_y, pred2))
    sheet.write(7, 4, recall_score(test_y, pred2))
    sheet.write(7, 5, f1_score(test_y, pred2))
    sheet.write(7, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    sheet.write(7, 7, auc)
    sheet.write(7, 8, fpr)
    sheet.write(7, 9, str(tp))
    sheet.write(7, 10, str(fn))
    sheet.write(7, 11, str(fp))
    sheet.write(7, 12, str(tn))
    sheet.write(7, 13, str(tot_n))
    sheet.write(7, 14, str(fptp_rate))

    precision2_3 = np.append(precision2_3, precision_score(test_y, pred2))
    recall2_3 = np.append(recall2_3, recall_score(test_y, pred2))
    f1_score2_3 = np.append(f1_score2_3, f1_score(test_y, pred2))
    kappa2_3 = np.append(kappa2_3, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    roc2_3 = np.append(roc2_3, auc)
    fpr2_3 = np.append(fpr2_3, fpr)

    ############
    pred2 = (probs2 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred2).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs2)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(8, 0, "Support Vector Machine")
    sheet.write(8, 1, "iraw_idur_OEF")
    sheet.write(8, 2, "0.03")
    sheet.write(8, 3, precision_score(test_y, pred2))
    sheet.write(8, 4, recall_score(test_y, pred2))
    sheet.write(8, 5, f1_score(test_y, pred2))
    sheet.write(8, 6, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    sheet.write(8, 7, auc)
    sheet.write(8, 8, fpr)
    sheet.write(8, 9, str(tp))
    sheet.write(8, 10, str(fn))
    sheet.write(8, 11, str(fp))
    sheet.write(8, 12, str(tn))
    sheet.write(8, 13, str(tot_n))
    sheet.write(8, 14, str(fptp_rate))

    precision2_4 = np.append(precision2_4, precision_score(test_y, pred2))
    recall2_4 = np.append(recall2_4, recall_score(test_y, pred2))
    f1_score2_4 = np.append(f1_score2_4, f1_score(test_y, pred2))
    kappa2_4 = np.append(kappa2_4, cohen_kappa_score(test_y, pred2, weights = 'quadratic'))
    roc2_4 = np.append(roc2_4, auc)
    fpr2_4 = np.append(fpr2_4, fpr)

###################################
    ###
    model3 = LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg")
    # Fit, Predict
    model3.fit(x_smote, y_smote)
    probs3 = model3.predict_proba(test_x)
    probs3 = probs3[:, 1]

    pred3 = (probs3 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs3)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(9, 0, "Logistic Regression")
    sheet.write(9, 1, "iraw_idur_OEF")
    sheet.write(9, 2, "0.7")
    sheet.write(9, 3, precision_score(test_y, pred3))
    sheet.write(9, 4, recall_score(test_y, pred3))
    sheet.write(9, 5, f1_score(test_y, pred3))
    sheet.write(9, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    sheet.write(9, 7, auc)
    sheet.write(9, 8, fpr)
    sheet.write(9, 9, str(tp))
    sheet.write(9, 10, str(fn))
    sheet.write(9, 11, str(fp))
    sheet.write(9, 12, str(tn))
    sheet.write(9, 13, str(tot_n))
    sheet.write(9, 14, str(fptp_rate))

    precision3_1 = np.append(precision3_1, precision_score(test_y, pred3))
    recall3_1 = np.append(recall3_1, recall_score(test_y, pred3))
    f1_score3_1 = np.append(f1_score3_1, f1_score(test_y, pred3))
    kappa3_1 = np.append(kappa3_1, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    roc3_1 = np.append(roc3_1, auc)
    fpr3_1 = np.append(fpr3_1, fpr)

    ######
    pred3 = (probs3 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs3)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(10, 0, "Logistic Regression")
    sheet.write(10, 1, "iraw_idur_OEF")
    sheet.write(10, 2, "0.5")
    sheet.write(10, 3, precision_score(test_y, pred3))
    sheet.write(10, 4, recall_score(test_y, pred3))
    sheet.write(10, 5, f1_score(test_y, pred3))
    sheet.write(10, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    sheet.write(10, 7, auc)
    sheet.write(10, 8, fpr)
    sheet.write(10, 9, str(tp))
    sheet.write(10, 10, str(fn))
    sheet.write(10, 11, str(fp))
    sheet.write(10, 12, str(tn))
    sheet.write(10, 13, str(tot_n))
    sheet.write(10, 14, str(fptp_rate))

    precision3_2 = np.append(precision3_2, precision_score(test_y, pred3))
    recall3_2 = np.append(recall3_2, recall_score(test_y, pred3))
    f1_score3_2 = np.append(f1_score3_2, f1_score(test_y, pred3))
    kappa3_2 = np.append(kappa3_2, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    roc3_2 = np.append(roc3_2, auc)
    fpr3_2 = np.append(fpr3_2, fpr)

    ######
    pred3 = (probs3 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs3)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(11, 0, "Logistic Regression")
    sheet.write(11, 1, "iraw_idur_OEF")
    sheet.write(11, 2, "0.97")
    sheet.write(11, 3, precision_score(test_y, pred3))
    sheet.write(11, 4, recall_score(test_y, pred3))
    sheet.write(11, 5, f1_score(test_y, pred3))
    sheet.write(11, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    sheet.write(11, 7, auc)
    sheet.write(11, 8, fpr)
    sheet.write(11, 9, str(tp))
    sheet.write(11, 10, str(fn))
    sheet.write(11, 11, str(fp))
    sheet.write(11, 12, str(tn))
    sheet.write(11, 13, str(tot_n))
    sheet.write(11, 14, str(fptp_rate))

    precision3_3 = np.append(precision3_3, precision_score(test_y, pred3))
    recall3_3 = np.append(recall3_3, recall_score(test_y, pred3))
    f1_score3_3 = np.append(f1_score3_3, f1_score(test_y, pred3))
    kappa3_3 = np.append(kappa3_3, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    roc3_3 = np.append(roc3_3, auc)
    fpr3_3 = np.append(fpr3_3, fpr)

    ######
    pred3 = (probs3 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred3).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs3)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(12, 0, "Logistic Regression")
    sheet.write(12, 1, "iraw_idur_OEF")
    sheet.write(12, 2, "0.03")
    sheet.write(12, 3, precision_score(test_y, pred3))
    sheet.write(12, 4, recall_score(test_y, pred3))
    sheet.write(12, 5, f1_score(test_y, pred3))
    sheet.write(12, 6, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    sheet.write(12, 7, auc)
    sheet.write(12, 8, fpr)
    sheet.write(12, 9, str(tp))
    sheet.write(12, 10, str(fn))
    sheet.write(12, 11, str(fp))
    sheet.write(12, 12, str(tn))
    sheet.write(12, 13, str(tot_n))
    sheet.write(12, 14, str(fptp_rate))

    precision3_4 = np.append(precision3_4, precision_score(test_y, pred3))
    recall3_4 = np.append(recall3_4, recall_score(test_y, pred3))
    f1_score3_4 = np.append(f1_score3_4, f1_score(test_y, pred3))
    kappa3_4 = np.append(kappa3_4, cohen_kappa_score(test_y, pred3, weights = 'quadratic'))
    roc3_4 = np.append(roc3_4, auc)
    fpr3_4 = np.append(fpr3_4, fpr)


#########################################
    ###
    model4 = LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen")
    # Fit, Predict
    model4.fit(x_smote, y_smote)
    probs4 = model4.predict_proba(test_x)
    probs4 = probs4[:, 1]

    pred4 = (probs4 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs4)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(13, 0, "Discriminant Analysis")
    sheet.write(13, 1, "iraw_idur_OEF")
    sheet.write(13, 2, "0.7")
    sheet.write(13, 3, precision_score(test_y, pred4))
    sheet.write(13, 4, recall_score(test_y, pred4))
    sheet.write(13, 5, f1_score(test_y, pred4))
    sheet.write(13, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    sheet.write(13, 7, auc)
    sheet.write(13, 8, fpr)
    sheet.write(13, 9, str(tp))
    sheet.write(13, 10, str(fn))
    sheet.write(13, 11, str(fp))
    sheet.write(13, 12, str(tn))
    sheet.write(13, 13, str(tot_n))
    sheet.write(13, 14, str(fptp_rate))

    precision4_1 = np.append(precision4_1, precision_score(test_y, pred4))
    recall4_1 = np.append(recall4_1, recall_score(test_y, pred4))
    f1_score4_1 = np.append(f1_score4_1, f1_score(test_y, pred4))
    kappa4_1 = np.append(kappa4_1, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    roc4_1 = np.append(roc4_1, auc)
    fpr4_1 = np.append(fpr4_1, fpr)

    ############
    pred4 = (probs4 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs4)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(14, 0, "Discriminant Analysis")
    sheet.write(14, 1, "iraw_idur_OEF")
    sheet.write(14, 2, "0.5")
    sheet.write(14, 3, precision_score(test_y, pred4))
    sheet.write(14, 4, recall_score(test_y, pred4))
    sheet.write(14, 5, f1_score(test_y, pred4))
    sheet.write(14, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    sheet.write(14, 7, auc)
    sheet.write(14, 8, fpr)
    sheet.write(14, 9, str(tp))
    sheet.write(14, 10, str(fn))
    sheet.write(14, 11, str(fp))
    sheet.write(14, 12, str(tn))
    sheet.write(14, 13, str(tot_n))
    sheet.write(14, 14, str(fptp_rate))

    precision4_2 = np.append(precision4_2, precision_score(test_y, pred4))
    recall4_2 = np.append(recall4_2, recall_score(test_y, pred4))
    f1_score4_2 = np.append(f1_score4_2, f1_score(test_y, pred4))
    kappa4_2 = np.append(kappa4_2, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    roc4_2 = np.append(roc4_2, auc)
    fpr4_2 = np.append(fpr4_2, fpr)

    ############
    pred4 = (probs4 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs4)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(15, 0, "Discriminant Analysis")
    sheet.write(15, 1, "iraw_idur_OEF")
    sheet.write(15, 2, "0.97")
    sheet.write(15, 3, precision_score(test_y, pred4))
    sheet.write(15, 4, recall_score(test_y, pred4))
    sheet.write(15, 5, f1_score(test_y, pred4))
    sheet.write(15, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    sheet.write(15, 7, auc)
    sheet.write(15, 8, fpr)
    sheet.write(15, 9, str(tp))
    sheet.write(15, 10, str(fn))
    sheet.write(15, 11, str(fp))
    sheet.write(15, 12, str(tn))
    sheet.write(15, 13, str(tot_n))
    sheet.write(15, 14, str(fptp_rate))

    precision4_3 = np.append(precision4_3, precision_score(test_y, pred4))
    recall4_3 = np.append(recall4_3, recall_score(test_y, pred4))
    f1_score4_3 = np.append(f1_score4_3, f1_score(test_y, pred4))
    kappa4_3 = np.append(kappa4_3, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    roc4_3 = np.append(roc4_3, auc)
    fpr4_3 = np.append(fpr4_3, fpr)

    ############
    pred4 = (probs4 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred4).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs4)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(16, 0, "Discriminant Analysis")
    sheet.write(16, 1, "iraw_idur_OEF")
    sheet.write(16, 2, "0.03")
    sheet.write(16, 3, precision_score(test_y, pred4))
    sheet.write(16, 4, recall_score(test_y, pred4))
    sheet.write(16, 5, f1_score(test_y, pred4))
    sheet.write(16, 6, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    sheet.write(16, 7, auc)
    sheet.write(16, 8, fpr)
    sheet.write(16, 9, str(tp))
    sheet.write(16, 10, str(fn))
    sheet.write(16, 11, str(fp))
    sheet.write(16, 12, str(tn))
    sheet.write(16, 13, str(tot_n))
    sheet.write(16, 14, str(fptp_rate))

    precision4_4 = np.append(precision4_4, precision_score(test_y, pred4))
    recall4_4 = np.append(recall4_4, recall_score(test_y, pred4))
    f1_score4_4 = np.append(f1_score4_4, f1_score(test_y, pred4))
    kappa4_4 = np.append(kappa4_4, cohen_kappa_score(test_y, pred4, weights = 'quadratic'))
    roc4_4 = np.append(roc4_4, auc)
    fpr4_4 = np.append(fpr4_4, fpr)

###############################################
    ###
    model5 = GaussianNB(var_smoothing=6.579332246575682e-08)
    # Fit, Predict
    model5.fit(x_smote, y_smote)
    probs5 = model5.predict_proba(test_x)
    probs5 = probs5[:, 1]

    pred5 = (probs5 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs5)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(17, 0, "Naive Bayes")
    sheet.write(17, 1, "iraw_idur_OEF")
    sheet.write(17, 2, "0.7")
    sheet.write(17, 3, precision_score(test_y, pred5))
    sheet.write(17, 4, recall_score(test_y, pred5))
    sheet.write(17, 5, f1_score(test_y, pred5))
    sheet.write(17, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    sheet.write(17, 7, auc)
    sheet.write(17, 8, fpr)
    sheet.write(17, 9, str(tp))
    sheet.write(17, 10, str(fn))
    sheet.write(17, 11, str(fp))
    sheet.write(17, 12, str(tn))
    sheet.write(17, 13, str(tot_n))
    sheet.write(17, 14, str(fptp_rate))

    precision5_1 = np.append(precision5_1, precision_score(test_y, pred5))
    recall5_1 = np.append(recall5_1, recall_score(test_y, pred5))
    f1_score5_1 = np.append(f1_score5_1, f1_score(test_y, pred5))
    kappa5_1 = np.append(kappa5_1, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    roc5_1 = np.append(roc5_1, auc)
    fpr5_1 = np.append(fpr5_1, fpr)


    ###############
    pred5 = (probs5 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs5)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(18, 0, "Naive Bayes")
    sheet.write(18, 1, "iraw_idur_OEF")
    sheet.write(18, 2, "0.5")
    sheet.write(18, 3, precision_score(test_y, pred5))
    sheet.write(18, 4, recall_score(test_y, pred5))
    sheet.write(18, 5, f1_score(test_y, pred5))
    sheet.write(18, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    sheet.write(18, 7, auc)
    sheet.write(18, 8, fpr)
    sheet.write(18, 9, str(tp))
    sheet.write(18, 10, str(fn))
    sheet.write(18, 11, str(fp))
    sheet.write(18, 12, str(tn))
    sheet.write(18, 13, str(tot_n))
    sheet.write(18, 14, str(fptp_rate))

    precision5_2 = np.append(precision5_2, precision_score(test_y, pred5))
    recall5_2 = np.append(recall5_2, recall_score(test_y, pred5))
    f1_score5_2 = np.append(f1_score5_2, f1_score(test_y, pred5))
    kappa5_2 = np.append(kappa5_2, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    roc5_2 = np.append(roc5_2, auc)
    fpr5_2 = np.append(fpr5_2, fpr)

    ###############
    pred5 = (probs5 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs5)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(19, 0, "Naive Bayes")
    sheet.write(19, 1, "iraw_idur_OEF")
    sheet.write(19, 2, "0.97")
    sheet.write(19, 3, precision_score(test_y, pred5))
    sheet.write(19, 4, recall_score(test_y, pred5))
    sheet.write(19, 5, f1_score(test_y, pred5))
    sheet.write(19, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    sheet.write(19, 7, auc)
    sheet.write(19, 8, fpr)
    sheet.write(19, 9, str(tp))
    sheet.write(19, 10, str(fn))
    sheet.write(19, 11, str(fp))
    sheet.write(19, 12, str(tn))
    sheet.write(19, 13, str(tot_n))
    sheet.write(19, 14, str(fptp_rate))

    precision5_3 = np.append(precision5_3, precision_score(test_y, pred5))
    recall5_3 = np.append(recall5_3, recall_score(test_y, pred5))
    f1_score5_3 = np.append(f1_score5_3, f1_score(test_y, pred5))
    kappa5_3 = np.append(kappa5_3, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    roc5_3 = np.append(roc5_3, auc)
    fpr5_3 = np.append(fpr5_3, fpr)

    ###############
    pred5 = (probs5 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred5).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs5)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(20, 0, "Naive Bayes")
    sheet.write(20, 1, "iraw_idur_OEF")
    sheet.write(20, 2, "0.03")
    sheet.write(20, 3, precision_score(test_y, pred5))
    sheet.write(20, 4, recall_score(test_y, pred5))
    sheet.write(20, 5, f1_score(test_y, pred5))
    sheet.write(20, 6, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    sheet.write(20, 7, auc)
    sheet.write(20, 8, fpr)
    sheet.write(20, 9, str(tp))
    sheet.write(20, 10, str(fn))
    sheet.write(20, 11, str(fp))
    sheet.write(20, 12, str(tn))
    sheet.write(20, 13, str(tot_n))
    sheet.write(20, 14, str(fptp_rate))

    precision5_4 = np.append(precision5_4, precision_score(test_y, pred5))
    recall5_4 = np.append(recall5_4, recall_score(test_y, pred5))
    f1_score5_4 = np.append(f1_score5_4, f1_score(test_y, pred5))
    kappa5_4 = np.append(kappa5_4, cohen_kappa_score(test_y, pred5, weights = 'quadratic'))
    roc5_4 = np.append(roc5_4, auc)
    fpr5_4 = np.append(fpr5_4, fpr)

##############################################################
    ###
    model6 = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")
    # Fit, Predict
    model6.fit(x_smote, y_smote)
    probs6 = model6.predict_proba(test_x)
    probs6 = probs6[:, 1]

    pred6 = (probs6 >= 0.7).astype(bool) # set threshold as 0.7

    tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs6)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(21, 0, "Neural Network")
    sheet.write(21, 1, "iraw_idur_OEF")
    sheet.write(21, 2, "0.7")
    sheet.write(21, 3, precision_score(test_y, pred6))
    sheet.write(21, 4, recall_score(test_y, pred6))
    sheet.write(21, 5, f1_score(test_y, pred6))
    sheet.write(21, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    sheet.write(21, 7, auc)
    sheet.write(21, 8, fpr)
    sheet.write(21, 9, str(tp))
    sheet.write(21, 10, str(fn))
    sheet.write(21, 11, str(fp))
    sheet.write(21, 12, str(tn))
    sheet.write(21, 13, str(tot_n))
    sheet.write(21, 14, str(fptp_rate))

    precision6_1 = np.append(precision6_1, precision_score(test_y, pred6))
    recall6_1 = np.append(recall6_1, recall_score(test_y, pred6))
    f1_score6_1 = np.append(f1_score6_1, f1_score(test_y, pred6))
    kappa6_1 = np.append(kappa6_1, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    roc6_1 = np.append(roc6_1, auc)
    fpr6_1 = np.append(fpr6_1, fpr)

    ##################
    pred6 = (probs6 >= 0.5).astype(bool)

    tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs6)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(22, 0, "Neural Network")
    sheet.write(22, 1, "iraw_idur_OEF")
    sheet.write(22, 2, "0.5")
    sheet.write(22, 3, precision_score(test_y, pred6))
    sheet.write(22, 4, recall_score(test_y, pred6))
    sheet.write(22, 5, f1_score(test_y, pred6))
    sheet.write(22, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    sheet.write(22, 7, auc)
    sheet.write(22, 8, fpr)
    sheet.write(22, 9, str(tp))
    sheet.write(22, 10, str(fn))
    sheet.write(22, 11, str(fp))
    sheet.write(22, 12, str(tn))
    sheet.write(22, 13, str(tot_n))
    sheet.write(22, 14, str(fptp_rate))

    precision6_2 = np.append(precision6_2, precision_score(test_y, pred6))
    recall6_2 = np.append(recall6_2, recall_score(test_y, pred6))
    f1_score6_2 = np.append(f1_score6_2, f1_score(test_y, pred6))
    kappa6_2 = np.append(kappa6_2, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    roc6_2 = np.append(roc6_2, auc)
    fpr6_2 = np.append(fpr6_2, fpr)

    ##################
    pred6 = (probs6 >= 0.97).astype(bool)

    tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs6)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(23, 0, "Neural Network")
    sheet.write(23, 1, "iraw_idur_OEF")
    sheet.write(23, 2, "0.97")
    sheet.write(23, 3, precision_score(test_y, pred6))
    sheet.write(23, 4, recall_score(test_y, pred6))
    sheet.write(23, 5, f1_score(test_y, pred6))
    sheet.write(23, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    sheet.write(23, 7, auc)
    sheet.write(23, 8, fpr)
    sheet.write(23, 9, str(tp))
    sheet.write(23, 10, str(fn))
    sheet.write(23, 11, str(fp))
    sheet.write(23, 12, str(tn))
    sheet.write(23, 13, str(tot_n))
    sheet.write(23, 14, str(fptp_rate))

    precision6_3 = np.append(precision6_3, precision_score(test_y, pred6))
    recall6_3 = np.append(recall6_3, recall_score(test_y, pred6))
    f1_score6_3 = np.append(f1_score6_3, f1_score(test_y, pred6))
    kappa6_3 = np.append(kappa6_3, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    roc6_3 = np.append(roc6_3, auc)
    fpr6_3 = np.append(fpr6_3, fpr)

    ##################
    pred6 = (probs6 >= 0.03).astype(bool)

    tn, fp, fn, tp = confusion_matrix(test_y, pred6).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs6)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(24, 0, "Neural Network")
    sheet.write(24, 1, "iraw_idur_OEF")
    sheet.write(24, 2, "0.03")
    sheet.write(24, 3, precision_score(test_y, pred6))
    sheet.write(24, 4, recall_score(test_y, pred6))
    sheet.write(24, 5, f1_score(test_y, pred6))
    sheet.write(24, 6, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    sheet.write(24, 7, auc)
    sheet.write(24, 8, fpr)
    sheet.write(24, 9, str(tp))
    sheet.write(24, 10, str(fn))
    sheet.write(24, 11, str(fp))
    sheet.write(24, 12, str(tn))
    sheet.write(24, 13, str(tot_n))
    sheet.write(24, 14, str(fptp_rate))

    precision6_4 = np.append(precision6_4, precision_score(test_y, pred6))
    recall6_4 = np.append(recall6_4, recall_score(test_y, pred6))
    f1_score6_4 = np.append(f1_score6_4, f1_score(test_y, pred6))
    kappa6_4 = np.append(kappa6_4, cohen_kappa_score(test_y, pred6, weights = 'quadratic'))
    roc6_4 = np.append(roc6_4, auc)
    fpr6_4 = np.append(fpr6_4, fpr)

############################################
    ###
    model7 = RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
    # Fit, Predict
    model7.fit(x_smote, y_smote)
    probs7 = model7.predict_proba(test_x)
    probs7 = probs7[:, 1]

    pred7 = (probs7 >= 0.7).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs7)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(25, 0, "Random Forest")
    sheet.write(25, 1, "iraw_idur_OEF")
    sheet.write(25, 2, "0.7")
    sheet.write(25, 3, precision_score(test_y, pred7))
    sheet.write(25, 4, recall_score(test_y, pred7))
    sheet.write(25, 5, f1_score(test_y, pred7))
    sheet.write(25, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    sheet.write(25, 7, auc)
    sheet.write(25, 8, fpr)
    sheet.write(25, 9, str(tp))
    sheet.write(25, 10, str(fn))
    sheet.write(25, 11, str(fp))
    sheet.write(25, 12, str(tn))
    sheet.write(25, 13, str(tot_n))
    sheet.write(25, 14, str(fptp_rate))

    precision7_1 = np.append(precision7_1, precision_score(test_y, pred7))
    recall7_1 = np.append(recall7_1, recall_score(test_y, pred7))
    f1_score7_1 = np.append(f1_score7_1, f1_score(test_y, pred7))
    kappa7_1 = np.append(kappa7_1, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    roc7_1 = np.append(roc7_1, auc)
    fpr7_1 = np.append(fpr7_1, fpr)

    ##################
    pred7 = (probs7 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs7)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(26, 0, "Random Forest")
    sheet.write(26, 1, "iraw_idur_OEF")
    sheet.write(26, 2, "0.5")
    sheet.write(26, 3, precision_score(test_y, pred7))
    sheet.write(26, 4, recall_score(test_y, pred7))
    sheet.write(26, 5, f1_score(test_y, pred7))
    sheet.write(26, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    sheet.write(26, 7, auc)
    sheet.write(26, 8, fpr)
    sheet.write(26, 9, str(tp))
    sheet.write(26, 10, str(fn))
    sheet.write(26, 11, str(fp))
    sheet.write(26, 12, str(tn))
    sheet.write(26, 13, str(tot_n))
    sheet.write(26, 14, str(fptp_rate))

    precision7_2 = np.append(precision7_2, precision_score(test_y, pred7))
    recall7_2 = np.append(recall7_2, recall_score(test_y, pred7))
    f1_score7_2 = np.append(f1_score7_2, f1_score(test_y, pred7))
    kappa7_2 = np.append(kappa7_2, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    roc7_2 = np.append(roc7_2, auc)
    fpr7_2 = np.append(fpr7_2, fpr)

    ##################
    pred7 = (probs7 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs7)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(27, 0, "Random Forest")
    sheet.write(27, 1, "iraw_idur_OEF")
    sheet.write(27, 2, "0.97")
    sheet.write(27, 3, precision_score(test_y, pred7))
    sheet.write(27, 4, recall_score(test_y, pred7))
    sheet.write(27, 5, f1_score(test_y, pred7))
    sheet.write(27, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    sheet.write(27, 7, auc)
    sheet.write(27, 8, fpr)
    sheet.write(27, 9, str(tp))
    sheet.write(27, 10, str(fn))
    sheet.write(27, 11, str(fp))
    sheet.write(27, 12, str(tn))
    sheet.write(27, 13, str(tot_n))
    sheet.write(27, 14, str(fptp_rate))

    precision7_3 = np.append(precision7_3, precision_score(test_y, pred7))
    recall7_3 = np.append(recall7_3, recall_score(test_y, pred7))
    f1_score7_3 = np.append(f1_score7_3, f1_score(test_y, pred7))
    kappa7_3 = np.append(kappa7_3, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    roc7_3 = np.append(roc7_3, auc)
    fpr7_3 = np.append(fpr7_3, fpr)

    ##################
    pred7 = (probs7 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred7).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs7)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(28, 0, "Random Forest")
    sheet.write(28, 1, "iraw_idur_OEF")
    sheet.write(28, 2, "0.03")
    sheet.write(28, 3, precision_score(test_y, pred7))
    sheet.write(28, 4, recall_score(test_y, pred7))
    sheet.write(28, 5, f1_score(test_y, pred7))
    sheet.write(28, 6, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    sheet.write(28, 7, auc)
    sheet.write(28, 8, fpr)
    sheet.write(28, 9, str(tp))
    sheet.write(28, 10, str(fn))
    sheet.write(28, 11, str(fp))
    sheet.write(28, 12, str(tn))
    sheet.write(28, 13, str(tot_n))
    sheet.write(28, 14, str(fptp_rate))

    precision7_4 = np.append(precision7_4, precision_score(test_y, pred7))
    recall7_4 = np.append(recall7_4, recall_score(test_y, pred7))
    f1_score7_4 = np.append(f1_score7_4, f1_score(test_y, pred7))
    kappa7_4 = np.append(kappa7_4, cohen_kappa_score(test_y, pred7, weights = 'quadratic'))
    roc7_4 = np.append(roc7_4, auc)
    fpr7_4 = np.append(fpr7_4, fpr)

#################################################
    ###
    model8 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)
    # Fit, Predict
    model8.fit(x_smote, y_smote)
    probs8 = model8.predict_proba(test_x)
    probs8 = probs8[:, 1]

    pred8 = (probs8 >= 0.7).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs8)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(29, 0, "Gradient Boosting")
    sheet.write(29, 1, "iraw_idur_OEF")
    sheet.write(29, 2, "0.7")
    sheet.write(29, 3, precision_score(test_y, pred8))
    sheet.write(29, 4, recall_score(test_y, pred8))
    sheet.write(29, 5, f1_score(test_y, pred8))
    sheet.write(29, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    sheet.write(29, 7, auc)
    sheet.write(29, 8, fpr)
    sheet.write(29, 9, str(tp))
    sheet.write(29, 10, str(fn))
    sheet.write(29, 11, str(fp))
    sheet.write(29, 12, str(tn))
    sheet.write(29, 13, str(tot_n))
    sheet.write(29, 14, str(fptp_rate))

    precision8_1 = np.append(precision8_1, precision_score(test_y, pred8))
    recall8_1 = np.append(recall8_1, recall_score(test_y, pred8))
    f1_score8_1 = np.append(f1_score8_1, f1_score(test_y, pred8))
    kappa8_1 = np.append(kappa8_1, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    roc8_1 = np.append(roc8_1, auc)
    fpr8_1 = np.append(fpr8_1, fpr)

    ######################
    pred8 = (probs8 >= 0.5).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs8)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(30, 0, "Gradient Boosting")
    sheet.write(30, 1, "iraw_idur_OEF")
    sheet.write(30, 2, "0.5")
    sheet.write(30, 3, precision_score(test_y, pred8))
    sheet.write(30, 4, recall_score(test_y, pred8))
    sheet.write(30, 5, f1_score(test_y, pred8))
    sheet.write(30, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    sheet.write(30, 7, auc)
    sheet.write(30, 8, fpr)
    sheet.write(30, 9, str(tp))
    sheet.write(30, 10, str(fn))
    sheet.write(30, 11, str(fp))
    sheet.write(30, 12, str(tn))
    sheet.write(30, 13, str(tot_n))
    sheet.write(30, 14, str(fptp_rate))

    precision8_2 = np.append(precision8_2, precision_score(test_y, pred8))
    recall8_2 = np.append(recall8_2, recall_score(test_y, pred8))
    f1_score8_2 = np.append(f1_score8_2, f1_score(test_y, pred8))
    kappa8_2 = np.append(kappa8_2, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    roc8_2 = np.append(roc8_2, auc)
    fpr8_2 = np.append(fpr8_2, fpr)

    ######################
    pred8 = (probs8 >= 0.97).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs8)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(31, 0, "Gradient Boosting")
    sheet.write(31, 1, "iraw_idur_OEF")
    sheet.write(31, 2, "0.97")
    sheet.write(31, 3, precision_score(test_y, pred8))
    sheet.write(31, 4, recall_score(test_y, pred8))
    sheet.write(31, 5, f1_score(test_y, pred8))
    sheet.write(31, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    sheet.write(31, 7, auc)
    sheet.write(31, 8, fpr)
    sheet.write(31, 9, str(tp))
    sheet.write(31, 10, str(fn))
    sheet.write(31, 11, str(fp))
    sheet.write(31, 12, str(tn))
    sheet.write(31, 13, str(tot_n))
    sheet.write(31, 14, str(fptp_rate))

    precision8_3 = np.append(precision8_3, precision_score(test_y, pred8))
    recall8_3 = np.append(recall8_3, recall_score(test_y, pred8))
    f1_score8_3 = np.append(f1_score8_3, f1_score(test_y, pred8))
    kappa8_3 = np.append(kappa8_3, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    roc8_3 = np.append(roc8_3, auc)
    fpr8_3 = np.append(fpr8_3, fpr)

    ######################
    pred8 = (probs8 >= 0.03).astype(bool) 

    tn, fp, fn, tp = confusion_matrix(test_y, pred8).ravel()
    fpr = (fp)/(fp+tn)
    auc = roc_auc_score(test_y, probs8)
    fptp_rate = (fp+tp)/(tn+fp+fn+tp)
    tot_n =  tn+fp+fn+tp

    #### write to excel sheet ####
    sheet.write(32, 0, "Gradient Boosting")
    sheet.write(32, 1, "iraw_idur_OEF")
    sheet.write(32, 2, "0.03")
    sheet.write(32, 3, precision_score(test_y, pred8))
    sheet.write(32, 4, recall_score(test_y, pred8))
    sheet.write(32, 5, f1_score(test_y, pred8))
    sheet.write(32, 6, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    sheet.write(32, 7, auc)
    sheet.write(32, 8, fpr)
    sheet.write(32, 9, str(tp))
    sheet.write(32, 10, str(fn))
    sheet.write(32, 11, str(fp))
    sheet.write(32, 12, str(tn))
    sheet.write(32, 13, str(tot_n))
    sheet.write(32, 14, str(fptp_rate))

    precision8_4 = np.append(precision8_4, precision_score(test_y, pred8))
    recall8_4 = np.append(recall8_4, recall_score(test_y, pred8))
    f1_score8_4 = np.append(f1_score8_4, f1_score(test_y, pred8))
    kappa8_4 = np.append(kappa8_4, cohen_kappa_score(test_y, pred8, weights = 'quadratic'))
    roc8_4 = np.append(roc8_4, auc)
    fpr8_4 = np.append(fpr8_4, fpr)


############
sheet_mean = wb.add_sheet('mean', cell_overwrite_ok=True) 

sheet_mean.write(0, 0, 'Model')
sheet_mean.write(0, 1, 'dataset')
sheet_mean.write(0, 2, 'threshold')
sheet_mean.write(0, 3, 'Precision')
sheet_mean.write(0, 4, 'Recall')
sheet_mean.write(0, 5, 'F1 Score')
sheet_mean.write(0, 6, 'weighted Kappa')
sheet_mean.write(0, 7, 'ROC AUC')
sheet_mean.write(0, 8, 'FPR')

sheet_mean.write(1, 0, "Decision Tree")
sheet_mean.write(1, 1, "iraw_idur_OEF")
sheet_mean.write(1, 2, "0.7")
sheet_mean.write(1, 3, np.mean(precision1_1))
sheet_mean.write(1, 4, np.mean(recall1_1))
sheet_mean.write(1, 5, np.mean(f1_score1_1))
sheet_mean.write(1, 6, np.mean(kappa1_1))
sheet_mean.write(1, 7, np.mean(roc1_1))
sheet_mean.write(1, 8, np.mean(fpr1_1))

sheet_mean.write(2, 0, "Decision Tree")
sheet_mean.write(2, 1, "iraw_idur_OEF")
sheet_mean.write(2, 2, "0.5")
sheet_mean.write(2, 3, np.mean(precision1_2))
sheet_mean.write(2, 4, np.mean(recall1_2))
sheet_mean.write(2, 5, np.mean(f1_score1_2))
sheet_mean.write(2, 6, np.mean(kappa1_2))
sheet_mean.write(2, 7, np.mean(roc1_2))
sheet_mean.write(2, 8, np.mean(fpr1_2))

sheet_mean.write(3, 0, "Decision Tree")
sheet_mean.write(3, 1, "iraw_idur_OEF")
sheet_mean.write(3, 2, "0.97")
sheet_mean.write(3, 3, np.mean(precision1_3))
sheet_mean.write(3, 4, np.mean(recall1_3))
sheet_mean.write(3, 5, np.mean(f1_score1_3))
sheet_mean.write(3, 6, np.mean(kappa1_3))
sheet_mean.write(3, 7, np.mean(roc1_3))
sheet_mean.write(3, 8, np.mean(fpr1_3))

sheet_mean.write(4, 0, "Decision Tree")
sheet_mean.write(4, 1, "iraw_idur_OEF")
sheet_mean.write(4, 2, "0.03")
sheet_mean.write(4, 3, np.mean(precision1_4))
sheet_mean.write(4, 4, np.mean(recall1_4))
sheet_mean.write(4, 5, np.mean(f1_score1_4))
sheet_mean.write(4, 6, np.mean(kappa1_4))
sheet_mean.write(4, 7, np.mean(roc1_4))
sheet_mean.write(4, 8, np.mean(fpr1_4))

###########
sheet_mean.write(5, 0, "Support Vector Machine")
sheet_mean.write(5, 1, "iraw_idur_OEF")
sheet_mean.write(5, 2, "0.7")
sheet_mean.write(5, 3, np.mean(precision2_1))
sheet_mean.write(5, 4, np.mean(recall2_1))
sheet_mean.write(5, 5, np.mean(f1_score2_1))
sheet_mean.write(5, 6, np.mean(kappa2_1))
sheet_mean.write(5, 7, np.mean(roc2_1))
sheet_mean.write(5, 8, np.mean(fpr2_1))

sheet_mean.write(6, 0, "Support Vector Machine")
sheet_mean.write(6, 1, "iraw_idur_OEF")
sheet_mean.write(6, 2, "0.5")
sheet_mean.write(6, 3, np.mean(precision2_2))
sheet_mean.write(6, 4, np.mean(recall2_2))
sheet_mean.write(6, 5, np.mean(f1_score2_2))
sheet_mean.write(6, 6, np.mean(kappa2_2))
sheet_mean.write(6, 7, np.mean(roc2_2))
sheet_mean.write(6, 8, np.mean(fpr2_2))

sheet_mean.write(7, 0, "Support Vector Machine")
sheet_mean.write(7, 1, "iraw_idur_OEF")
sheet_mean.write(7, 2, "0.97")
sheet_mean.write(7, 3, np.mean(precision2_3))
sheet_mean.write(7, 4, np.mean(recall2_3))
sheet_mean.write(7, 5, np.mean(f1_score2_3))
sheet_mean.write(7, 6, np.mean(kappa2_3))
sheet_mean.write(7, 7, np.mean(roc2_3))
sheet_mean.write(7, 8, np.mean(fpr2_3))

sheet_mean.write(8, 0, "Support Vector Machine")
sheet_mean.write(8, 1, "iraw_idur_OEF")
sheet_mean.write(8, 2, "0.03")
sheet_mean.write(8, 3, np.mean(precision2_4))
sheet_mean.write(8, 4, np.mean(recall2_4))
sheet_mean.write(8, 5, np.mean(f1_score2_4))
sheet_mean.write(8, 6, np.mean(kappa2_4))
sheet_mean.write(8, 7, np.mean(roc2_4))
sheet_mean.write(8, 8, np.mean(fpr2_4))

######################
sheet_mean.write(9, 0, "Logistic Regression")
sheet_mean.write(9, 1, "iraw_idur_OEF")
sheet_mean.write(9, 2, "0.7")
sheet_mean.write(9, 3, np.mean(precision3_1))
sheet_mean.write(9, 4, np.mean(recall3_1))
sheet_mean.write(9, 5, np.mean(f1_score3_1))
sheet_mean.write(9, 6, np.mean(kappa3_1))
sheet_mean.write(9, 7, np.mean(roc3_1))
sheet_mean.write(9, 8, np.mean(fpr3_1))

sheet_mean.write(10, 0, "Logistic Regression")
sheet_mean.write(10, 1, "iraw_idur_OEF")
sheet_mean.write(10, 2, "0.5")
sheet_mean.write(10, 3, np.mean(precision3_2))
sheet_mean.write(10, 4, np.mean(recall3_2))
sheet_mean.write(10, 5, np.mean(f1_score3_2))
sheet_mean.write(10, 6, np.mean(kappa3_2))
sheet_mean.write(10, 7, np.mean(roc3_2))
sheet_mean.write(10, 8, np.mean(fpr3_2))

sheet_mean.write(11, 0, "Logistic Regression")
sheet_mean.write(11, 1, "iraw_idur_OEF")
sheet_mean.write(11, 2, "0.97")
sheet_mean.write(11, 3, np.mean(precision3_3))
sheet_mean.write(11, 4, np.mean(recall3_3))
sheet_mean.write(11, 5, np.mean(f1_score3_3))
sheet_mean.write(11, 6, np.mean(kappa3_3))
sheet_mean.write(11, 7, np.mean(roc3_3))
sheet_mean.write(11, 8, np.mean(fpr3_3))

sheet_mean.write(12, 0, "Logistic Regression")
sheet_mean.write(12, 1, "iraw_idur_OEF")
sheet_mean.write(12, 2, "0.03")
sheet_mean.write(12, 3, np.mean(precision3_4))
sheet_mean.write(12, 4, np.mean(recall3_4))
sheet_mean.write(12, 5, np.mean(f1_score3_4))
sheet_mean.write(12, 6, np.mean(kappa3_4))
sheet_mean.write(12, 7, np.mean(roc3_4))
sheet_mean.write(12, 8, np.mean(fpr3_4))


##################################
sheet_mean.write(13, 0, "Discriminant Analysis")
sheet_mean.write(13, 1, "iraw_idur_OEF")
sheet_mean.write(13, 2, "0.7")
sheet_mean.write(13, 3, np.mean(precision4_1))
sheet_mean.write(13, 4, np.mean(recall4_1))
sheet_mean.write(13, 5, np.mean(f1_score4_1))
sheet_mean.write(13, 6, np.mean(kappa4_1))
sheet_mean.write(13, 7, np.mean(roc4_1))
sheet_mean.write(13, 8, np.mean(fpr4_1))

sheet_mean.write(14, 0, "Discriminant Analysis")
sheet_mean.write(14, 1, "iraw_idur_OEF")
sheet_mean.write(14, 2, "0.5")
sheet_mean.write(14, 3, np.mean(precision4_2))
sheet_mean.write(14, 4, np.mean(recall4_2))
sheet_mean.write(14, 5, np.mean(f1_score4_2))
sheet_mean.write(14, 6, np.mean(kappa4_2))
sheet_mean.write(14, 7, np.mean(roc4_2))
sheet_mean.write(14, 8, np.mean(fpr4_2))

sheet_mean.write(15, 0, "Discriminant Analysis")
sheet_mean.write(15, 1, "iraw_idur_OEF")
sheet_mean.write(15, 2, "0.97")
sheet_mean.write(15, 3, np.mean(precision4_3))
sheet_mean.write(15, 4, np.mean(recall4_3))
sheet_mean.write(15, 5, np.mean(f1_score4_3))
sheet_mean.write(15, 6, np.mean(kappa4_3))
sheet_mean.write(15, 7, np.mean(roc4_3))
sheet_mean.write(15, 8, np.mean(fpr4_3))

sheet_mean.write(16, 0, "Discriminant Analysis")
sheet_mean.write(16, 1, "iraw_idur_OEF")
sheet_mean.write(16, 2, "0.03")
sheet_mean.write(16, 3, np.mean(precision4_4))
sheet_mean.write(16, 4, np.mean(recall4_4))
sheet_mean.write(16, 5, np.mean(f1_score4_4))
sheet_mean.write(16, 6, np.mean(kappa4_4))
sheet_mean.write(16, 7, np.mean(roc4_4))
sheet_mean.write(16, 8, np.mean(fpr4_4))


#####################################
sheet_mean.write(17, 0, "Naive Bayes")
sheet_mean.write(17, 1, "iraw_idur_OEF")
sheet_mean.write(17, 2, "0.7")
sheet_mean.write(17, 3, np.mean(precision5_1))
sheet_mean.write(17, 4, np.mean(recall5_1))
sheet_mean.write(17, 5, np.mean(f1_score5_1))
sheet_mean.write(17, 6, np.mean(kappa5_1))
sheet_mean.write(17, 7, np.mean(roc5_1))
sheet_mean.write(17, 8, np.mean(fpr5_1))

sheet_mean.write(18, 0, "Naive Bayes")
sheet_mean.write(18, 1, "iraw_idur_OEF")
sheet_mean.write(18, 2, "0.5")
sheet_mean.write(18, 3, np.mean(precision5_2))
sheet_mean.write(18, 4, np.mean(recall5_2))
sheet_mean.write(18, 5, np.mean(f1_score5_2))
sheet_mean.write(18, 6, np.mean(kappa5_2))
sheet_mean.write(18, 7, np.mean(roc5_2))
sheet_mean.write(18, 8, np.mean(fpr5_2))

sheet_mean.write(19, 0, "Naive Bayes")
sheet_mean.write(19, 1, "iraw_idur_OEF")
sheet_mean.write(19, 2, "0.97")
sheet_mean.write(19, 3, np.mean(precision5_3))
sheet_mean.write(19, 4, np.mean(recall5_3))
sheet_mean.write(19, 5, np.mean(f1_score5_3))
sheet_mean.write(19, 6, np.mean(kappa5_3))
sheet_mean.write(19, 7, np.mean(roc5_3))
sheet_mean.write(19, 8, np.mean(fpr5_3))

sheet_mean.write(20, 0, "Naive Bayes")
sheet_mean.write(20, 1, "iraw_idur_OEF")
sheet_mean.write(20, 2, "0.03")
sheet_mean.write(20, 3, np.mean(precision5_4))
sheet_mean.write(20, 4, np.mean(recall5_4))
sheet_mean.write(20, 5, np.mean(f1_score5_4))
sheet_mean.write(20, 6, np.mean(kappa5_4))
sheet_mean.write(20, 7, np.mean(roc5_4))
sheet_mean.write(20, 8, np.mean(fpr5_4))


#########################################
sheet_mean.write(21, 0, "Neural Network")
sheet_mean.write(21, 1, "iraw_idur_OEF")
sheet_mean.write(21, 2, "0.7")
sheet_mean.write(21, 3, np.mean(precision6_1))
sheet_mean.write(21, 4, np.mean(recall6_1))
sheet_mean.write(21, 5, np.mean(f1_score6_1))
sheet_mean.write(21, 6, np.mean(kappa6_1))
sheet_mean.write(21, 7, np.mean(roc6_1))
sheet_mean.write(21, 8, np.mean(fpr6_1))

sheet_mean.write(22, 0, "Neural Network")
sheet_mean.write(22, 1, "iraw_idur_OEF")
sheet_mean.write(22, 2, "0.5")
sheet_mean.write(22, 3, np.mean(precision6_2))
sheet_mean.write(22, 4, np.mean(recall6_2))
sheet_mean.write(22, 5, np.mean(f1_score6_2))
sheet_mean.write(22, 6, np.mean(kappa6_2))
sheet_mean.write(22, 7, np.mean(roc6_2))
sheet_mean.write(22, 8, np.mean(fpr6_2))

sheet_mean.write(23, 0, "Neural Network")
sheet_mean.write(23, 1, "iraw_idur_OEF")
sheet_mean.write(23, 2, "0.97")
sheet_mean.write(23, 3, np.mean(precision6_3))
sheet_mean.write(23, 4, np.mean(recall6_3))
sheet_mean.write(23, 5, np.mean(f1_score6_3))
sheet_mean.write(23, 6, np.mean(kappa6_3))
sheet_mean.write(23, 7, np.mean(roc6_3))
sheet_mean.write(23, 8, np.mean(fpr6_3))

sheet_mean.write(24, 0, "Neural Network")
sheet_mean.write(24, 1, "iraw_idur_OEF")
sheet_mean.write(24, 2, "0.03")
sheet_mean.write(24, 3, np.mean(precision6_4))
sheet_mean.write(24, 4, np.mean(recall6_4))
sheet_mean.write(24, 5, np.mean(f1_score6_4))
sheet_mean.write(24, 6, np.mean(kappa6_4))
sheet_mean.write(24, 7, np.mean(roc6_4))
sheet_mean.write(24, 8, np.mean(fpr6_4))

#####################################
sheet_mean.write(25, 0, "Random Forest")
sheet_mean.write(25, 1, "iraw_idur_OEF")
sheet_mean.write(25, 2, "0.7")
sheet_mean.write(25, 3, np.mean(precision7_1))
sheet_mean.write(25, 4, np.mean(recall7_1))
sheet_mean.write(25, 5, np.mean(f1_score7_1))
sheet_mean.write(25, 6, np.mean(kappa7_1))
sheet_mean.write(25, 7, np.mean(roc7_1))
sheet_mean.write(25, 8, np.mean(fpr7_1))

sheet_mean.write(26, 0, "Random Forest")
sheet_mean.write(26, 1, "iraw_idur_OEF")
sheet_mean.write(26, 2, "0.5")
sheet_mean.write(26, 3, np.mean(precision7_2))
sheet_mean.write(26, 4, np.mean(recall7_2))
sheet_mean.write(26, 5, np.mean(f1_score7_2))
sheet_mean.write(26, 6, np.mean(kappa7_2))
sheet_mean.write(26, 7, np.mean(roc7_2))
sheet_mean.write(26, 8, np.mean(fpr7_2))

sheet_mean.write(27, 0, "Random Forest")
sheet_mean.write(27, 1, "iraw_idur_OEF")
sheet_mean.write(27, 2, "0.97")
sheet_mean.write(27, 3, np.mean(precision7_3))
sheet_mean.write(27, 4, np.mean(recall7_3))
sheet_mean.write(27, 5, np.mean(f1_score7_3))
sheet_mean.write(27, 6, np.mean(kappa7_3))
sheet_mean.write(27, 7, np.mean(roc7_3))
sheet_mean.write(27, 8, np.mean(fpr7_3))

sheet_mean.write(28, 0, "Random Forest")
sheet_mean.write(28, 1, "iraw_idur_OEF")
sheet_mean.write(28, 2, "0.03")
sheet_mean.write(28, 3, np.mean(precision7_4))
sheet_mean.write(28, 4, np.mean(recall7_4))
sheet_mean.write(28, 5, np.mean(f1_score7_4))
sheet_mean.write(28, 6, np.mean(kappa7_4))
sheet_mean.write(28, 7, np.mean(roc7_4))
sheet_mean.write(28, 8, np.mean(fpr7_4))

####################################
sheet_mean.write(29, 0, "Gradient Boosting")
sheet_mean.write(29, 1, "iraw_idur_OEF")
sheet_mean.write(29, 2, "0.7")
sheet_mean.write(29, 3, np.mean(precision8_1))
sheet_mean.write(29, 4, np.mean(recall8_1))
sheet_mean.write(29, 5, np.mean(f1_score8_1))
sheet_mean.write(29, 6, np.mean(kappa8_1))
sheet_mean.write(29, 7, np.mean(roc8_1))
sheet_mean.write(29, 8, np.mean(fpr8_1))

sheet_mean.write(30, 0, "Gradient Boosting")
sheet_mean.write(30, 1, "iraw_idur_OEF")
sheet_mean.write(30, 2, "0.5")
sheet_mean.write(30, 3, np.mean(precision8_2))
sheet_mean.write(30, 4, np.mean(recall8_2))
sheet_mean.write(30, 5, np.mean(f1_score8_2))
sheet_mean.write(30, 6, np.mean(kappa8_2))
sheet_mean.write(30, 7, np.mean(roc8_2))
sheet_mean.write(30, 8, np.mean(fpr8_2))

sheet_mean.write(31, 0, "Gradient Boosting")
sheet_mean.write(31, 1, "iraw_idur_OEF")
sheet_mean.write(31, 2, "0.97")
sheet_mean.write(31, 3, np.mean(precision8_3))
sheet_mean.write(31, 4, np.mean(recall8_3))
sheet_mean.write(31, 5, np.mean(f1_score8_3))
sheet_mean.write(31, 6, np.mean(kappa8_3))
sheet_mean.write(31, 7, np.mean(roc8_3))
sheet_mean.write(31, 8, np.mean(fpr8_3))

sheet_mean.write(32, 0, "Gradient Boosting")
sheet_mean.write(32, 1, "iraw_idur_OEF")
sheet_mean.write(32, 2, "0.03")
sheet_mean.write(32, 3, np.mean(precision8_4))
sheet_mean.write(32, 4, np.mean(recall8_4))
sheet_mean.write(32, 5, np.mean(f1_score8_4))
sheet_mean.write(32, 6, np.mean(kappa8_4))
sheet_mean.write(32, 7, np.mean(roc8_4))
sheet_mean.write(32, 8, np.mean(fpr8_4))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zer

In [19]:
wb.save('2forms_eachfold_undersampler33.xls') 